In [1]:
import torch
import math
import numpy as np
from matplotlib import pyplot as plt
import os
import sys
sys.path.append('..')
import ROOT
from Tools import syncer 
from Tools import user
from Tools import helpers
import itertools

Welcome to JupyROOT 6.24/06


In [2]:
class NN:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination: self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        '''
        Creates the Neural Network Architecture
        '''
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables). Returns the output of the NNs of dimensions: (noutput,ndatapoints)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        
        output=torch.zeros((noutputs,ndatapoints))
        for i in range(noutputs):
            x=self.n_hat[self.combination_list[i]](features)
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=torch.flatten(x)            
        return output
        
    
    def predict_r_hat(self, features, base_points):
        '''
        Evaluate positive xsec ratio for given theta and.
        First it fills the coefficients of the matrix containing the upper cholesky decomposition.
        Then it computes the multiplication of this matrix with the basepoints and squares it and sums it.
        '''
        ndatapoints=features.shape[0]
        output_NN = self.evaluate_NN(features)
        n_terms=len(self.coefficient_names)
        row,column=np.triu_indices(n_terms+1)
        Omega=torch.zeros((ndatapoints,n_terms+1,n_terms+1))
        for i in range(0, len(row)):
            Omega[:][row[i]][column[i]]=output_NN[i,:]
        out=torch.matmul(Omega_swapped,torch.transpose(base_points,0,1))
        return torch.linalg.norm(out, 2, 1)
    
    def predict_r_hat2(self, predictions,eft):
        return torch.add( 
        torch.sum( torch.stack( [(1. + predictions[(c,)]*eft[c])**2 for c in coefficients ]), dim=0),
        torch.sum( torch.stack( [torch.sum( torch.stack( [ predictions[(c_1,c_2)]*eft[c_2] for c_2 in coefficients[i_c_1:] ]), dim=0)**2 for i_c_1, c_1 in enumerate(coefficients) ] ), dim=0 ) )   

    
    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [3]:
def make_weight_ratio(weights, **kwargs):
    eft      = kwargs
    result = torch.ones(len(weights[()])) 
    for combination in combinations:
        if len(combination)==1:
            result += eft[combination[0]]*weights[combination]/weights[()]
        elif len(combination)==2:# add up without the factor 1/2 because off diagonals are only summed in upper triangle 
            result += (0.5 if len(set(combination))==1 else 1.)*eft[combination[0]]*eft[combination[1]]*weights[combination]/weights[()]
    return result

In [4]:
# loss functional
def f_loss(r_NN, features, predictions, base_points):
    loss = -0.5*weights[()].sum()
    for i_base_point, base_point in enumerate(base_points):
        fhat  = 1./(1. + r_NN.predict_r_hat2(predictions, base_point))
        loss += ( torch.tensor(weights[()])*( -0.25 + base_point_weight_ratios[i_base_point]*fhat**2 + (1-fhat)**2 ) ).sum()
    return loss

In [5]:
plot_directory="9_7_2022"
#coefficients=['cHW']
#coefficients=['cHW', 'cHWtil']
#coefficients=['cHW', 'cHWtil', 'cHQ3']
nEvents=30000
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 10000
plot_every    = 100

In [6]:
# training data
import ZH_Nakamura

ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

nfeatures = len(features[0]) 
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())


#pT=features[:,feature_names.index('pT')]
w0      = torch.from_numpy(weights[()])
wp      = torch.from_numpy(weights[('cHW',)]).float().to(device)
wpp     = torch.from_numpy(weights[('cHW','cHW')]).float().to(device)


#for key,value in weights.items():
 #   value*=bias_factor
  #  weights[key]=value

Requested 30000 events. Simulated 30000 events and 30000 survive pT_min cut of 0.


In [7]:
features = torch.from_numpy(features).float().to(device)

coefficients   = ['cHW']
combinations   =  [ ('cHW',), ('cHW', 'cHW')]
base_points = [ {'cHW':value} for value in [-1.5, -.8, -.4, -.2, .2, .4, .8, 1.5] ]

#coefficients   =  ( 'cHW', 'cHWtil') 
#combinations   =  [ ('cHW',), ('cHWtil',), ('cHW','cHW'), ('cHWtil','cHWtil'), ('cHW','cHWtil')]
#base_points = [ {'cHW':value1, 'cHWtil':value2} for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5]]

#coefficients   =  ( 'cHW', 'cHWtil', 'cHQ3') 
#combinations   =  [ ('cHW',), ('cHWtil',), ('cHQ3',), ('cHW', 'cHW'), ('cHWtil', 'cHWtil'), ('cHQ3', 'cHQ3'), ('cHW', 'cHWtil'), ('cHW', 'cHQ3'), ('cHWtil', 'cHQ3')] 
#base_points = list(filter( (lambda point: all([ coeff in args.coefficients or (not (coeff in point.keys() and point[coeff]!=0)) for coeff in point.keys()]) and any(map(bool, point.values()))), base_points)) 

coefficients = tuple(filter( lambda coeff: coeff in coefficients, list(coefficients))) 
combinations = tuple(filter( lambda comb: all([c in coefficients for c in comb]), combinations)) 

base_points    = list(map( lambda b:ZH_Nakamura.make_eft(**b), base_points ))

In [8]:
r_NN=NN(nfeatures,coefficients)

In [9]:
base_point_weight_ratios = list( map( lambda base_point: make_weight_ratio( weights, **base_point ), base_points ) )

In [10]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(sum([list(model.parameters()) for model in r_NN.n_hat.values()],[]), lr=learning_rate)

losses = []

tex = ROOT.TLatex()
tex.SetNDC()
tex.SetTextSize(0.04)

In [11]:
for nn in r_NN.n_hat.values():
    nn.train()

for epoch in range(n_epoch):
    # Forward pass: compute predicted y by passing x to the model.
    predictions = {combination:r_NN.n_hat[combination](features).squeeze() for combination in combinations}

    # Compute and print loss.
    loss = f_loss(r_NN,features, predictions,base_points)
    losses.append(loss.item())
    if epoch % 100 == 99:
        print("epoch", epoch, "loss",  loss.item())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if (epoch % plot_every)==0:
        with torch.no_grad():
            pred_t = predictions[('cHW',)].squeeze().cpu().detach().numpy()
            pred_s = predictions[('cHW','cHW')].squeeze().cpu().detach().numpy()
            
            #print("epoch", epoch, "loss inside loop",  loss_train.item())
            #print("epoch", epoch, "test loss",  loss_test.item())
            
            for var in plot_vars:
                binning     = plot_options[var]['binning']
                np_binning  = np.linspace(binning[1], binning[2], 1+binning[0])

                truth_0  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0 )
                truth_p  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=wp )
                truth_pp = np.histogram(features[:,feature_names.index(var)], np_binning, weights=wpp )

                pred_0  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0 )
                pred_p  = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0*2*pred_t )
                pred_pp = np.histogram(features[:,feature_names.index(var)], np_binning, weights=w0*2*(pred_t**2+pred_s**2) )

                h_yield       = helpers.make_TH1F(truth_0)
                h_truth_p     = helpers.make_TH1F(truth_p)
                h_truth_p     .Divide(h_yield) 
                h_truth_pp    = helpers.make_TH1F(truth_pp)
                h_truth_pp    .Divide(h_yield) 

                h_pred_p      = helpers.make_TH1F(pred_p)
                h_pred_p      .Divide(h_yield) 
                h_pred_pp     = helpers.make_TH1F(pred_pp)
                h_pred_pp     .Divide(h_yield) 

                l = ROOT.TLegend(0.3,0.7,0.9,0.95)
                l.SetNColumns(2)
                l.SetFillStyle(0)
                l.SetShadowColor(ROOT.kWhite)
                l.SetBorderSize(0)

                h_yield      .SetLineColor(ROOT.kGray+2) 
                h_truth_p    .SetLineColor(ROOT.kBlue) 
                h_truth_pp   .SetLineColor(ROOT.kRed) 
                h_pred_p     .SetLineColor(ROOT.kBlue) 
                h_pred_pp    .SetLineColor(ROOT.kRed) 
                h_yield      .SetMarkerColor(ROOT.kGray+2) 
                h_truth_p    .SetMarkerColor(ROOT.kBlue) 
                h_truth_pp   .SetMarkerColor(ROOT.kRed) 
                h_pred_p     .SetMarkerColor(ROOT.kBlue) 
                h_pred_pp    .SetMarkerColor(ROOT.kRed) 
                h_yield      .SetMarkerStyle(0)
                h_truth_p    .SetMarkerStyle(0)
                h_truth_pp   .SetMarkerStyle(0)
                h_pred_p     .SetMarkerStyle(0)
                h_pred_pp    .SetMarkerStyle(0)

                l.AddEntry(h_truth_p   , "1^{st.} der (truth)" ) 
                l.AddEntry(h_truth_pp  , "2^{st.} der (truth)" ) 
                l.AddEntry(h_pred_p    , "1^{st.} der (pred)" ) 
                l.AddEntry(h_pred_pp   , "2^{st.} der (pred)" ) 
                l.AddEntry(h_yield     , "yield" ) 

                h_truth_p    .SetLineStyle(ROOT.kDashed) 
                h_truth_pp   .SetLineStyle(ROOT.kDashed)

                lines = [ 
                        (0.16, 0.965, 'Epoch %5i    Loss %6.4f'%( epoch, loss ))
                        ]

                max_ = max( map( lambda h:h.GetMaximum(), [ h_truth_p, h_truth_pp ] ))

                h_yield.Scale(max_/h_yield.GetMaximum())
                for logY in [True, False]:
                    c1 = ROOT.TCanvas()
                    h_yield   .Draw("hist")
                    h_yield   .GetYaxis().SetRangeUser(0.1 if logY else 0, 10**(1.5)*max_ if logY else 1.5*max_)
                    h_yield   .Draw("hist")
                    h_truth_p .Draw("hsame") 
                    h_truth_pp.Draw("hsame")
                    h_pred_p  .Draw("hsame") 
                    h_pred_pp .Draw("hsame")
                    c1.SetLogy(logY) 
                    l.Draw()

                    drawObjects = [ tex.DrawLatex(*line) for line in lines ]
                    for o in drawObjects:
                        o.Draw()

                    plot_directory_final = os.path.join(plot_directory, "log" if logY else "lin")
                    helpers.copyIndexPHP( plot_directory )
                    c1.Print( os.path.join( plot_directory, "epoch_%05i_%s.png"%(epoch, var) ) )

epoch 99 loss 0.22323084640246588
epoch 199 loss 0.22120081793202898
epoch 299 loss 0.21931696421773922
epoch 399 loss 0.21612147190180436
epoch 499 loss 0.21195352488444558
epoch 599 loss 0.20908889177587342
epoch 699 loss 0.20885608686848564
epoch 799 loss 0.20866765606715992
epoch 899 loss 0.2082901461318729
epoch 999 loss 0.20764396427865567
epoch 1099 loss 0.20706681321397613
epoch 1199 loss 0.20634147039053372
epoch 1299 loss 0.2059323566390681
epoch 1399 loss 0.20551235374636076
epoch 1499 loss 0.20508697647862778
epoch 1599 loss 0.204847288024382
epoch 1699 loss 0.2047286631173549
epoch 1799 loss 0.2046453061198966
epoch 1899 loss 0.20459736743988788
epoch 1999 loss 0.20456190271736444
epoch 2099 loss 0.20452906891408343
epoch 2199 loss 0.2044978233647224
epoch 2299 loss 0.20447594143933612
epoch 2399 loss 0.2044546271743356
epoch 2499 loss 0.2044310153320849
epoch 2599 loss 0.20442231935642255
epoch 2699 loss 0.2044028668484032
epoch 2799 loss 0.20439497532628953
epoch 2899 lo

Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: png file 9_7_2022/epoch_00000_sqrt_s_hat.png has been created
Info in <TCanvas::Print>: png file 9_7_2022/epoch_00000_sqrt_s_hat.png has been created
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h (Potential memory leak).
Info in <TCanvas::Print>: png file 9_7_2022/epoch_00000_pT.png has been created
Info in <TCanvas::Print>: png fil